### Кредитный скоринг по модели. Ноутбук устарел . Можно стереть наверное

In [1]:
import cPickle
from pyspark import SparkConf, SparkContext, HiveContext
import re, sys, os
import numpy as np
import pandas as pd
import datetime
import cPickle
from pylightgbm.models import GBMClassifier
#from sklearn.feature_extraction import DictVectorizer
#import sklearn, sklearn.cross_validation
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#from pyspark.mllib.linalg import SparseVector
#from pyspark.mllib.evaluation import BinaryClassificationMetrics
#import hashlib
#from collections import Counter

In [ ]:
print('{0}: cred scoring started.').format(datetime.datetime.now())
sys.stdout.flush()

HIVE_HOST = 'ds-hadoop-cs01p'
HIVE_PORT = 10000
HIVE_USER = 'bigdatasys'
CONF={'hive.vectorized.execution.enabled':'true'
    ,'mapreduce.map.memory.mb':'4096'
    ,'mapreduce.map.child.java.opts':'-Xmx4g'
    ,'mapreduce.task.io.sort.mb':'1024'
    ,'mapreduce.reduce.child.java.opts':'-Xmx4g'
    ,'mapreduce.reduce.memory.mb':'7000'
    ,'mapreduce.reduce.shuffle.input.buffer.percent':'0.5'
    ,'mapreduce.input.fileinputformat.split.minsize':'536870912'
    ,'mapreduce.input.fileinputformat.split.maxsize':'1073741824'
    ,'hive.optimize.ppd':'true'
    ,'hive.merge.smallfiles.avgsize':'536870912'
    ,'hive.merge.mapredfiles':'true'
    ,'hive.merge.mapfiles':'true'
    ,'hive.hadoop.supports.splittable.combineinputformat':'true'
    ,'hive.exec.reducers.bytes.per.reducer':'536870912'
    ,'hive.exec.parallel':'true'
    ,'hive.exec.max.created.files':'10000000'
    ,'hive.exec.compress.output':'true'
    ,'hive.exec.dynamic.partition.mode':'nonstrict'
    ,'hive.exec.max.dynamic.partitions':'1000000'
    ,'hive.exec.max.dynamic.partitions.pernode':'100000'
    ,'io.seqfile.compression.type':'BLOCK'
    ,'mapreduce.map.failures.maxpercent':'5'
          }

from pyhive import hive
conn = hive.Connection(host=HIVE_HOST, port=HIVE_PORT, username=HIVE_USER, configuration=CONF)
cursor = conn.cursor()

In [2]:
hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''

sc.stop()
conf = (SparkConf()
        .set("spark.executor.instances", 16)
        .set("spark.driver.maxResultSize", "12g")
        .set('spark.driver.memory','12g')
        .set("spark.executor.memory", '8g')
        .set("spark.yarn.executor.memoryOverhead", 1048)
       )
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

for q in hive_config_query.split(';'):
    try:
        hc.sql(q)
    except:
        pass

In [12]:
# to delete
def add_months(ymd,months):
    '''ymd is datetime.datetime.'''
    year = ymd.year + (ymd.month + months - 1) / 12
    month = (ymd.month + months - 1) % 12 + 1
    day = ymd.day
    return datetime.datetime(year,month,day)


def get_last_features_date(cursor, first_calc_ymd, last_calc_ymd):
    cursor.execute('''select 1 from prod_features_liveinternet.phone_x_feature 
                  where first_calc_ymd = "#first_calc_ymd" and last_calc_ymd = "#last_calc_ymd"'''
               .replace( "#first_calc_ymd",first_calc_ymd.strftime('%Y-%m-%d'))
               .replace( "#last_calc_ymd",last_calc_ymd.strftime('%Y-%m-%d'))
            ) 
    return not len(cursor.fetchone()) > 0

def main():
    
    acc_months = 2
    ymd_to_calc   = datetime.datetime.combine(datetime.date.today(),datetime.time(0,0,0)) - datetime.timedelta(days = 2) #!!!!!!!!!!!!!!!!!!   # yesterday with 0 time 
    first_calc_ymd = add_months(datetime.datetime(ymd_to_calc.year,ymd_to_calc.month,1), - acc_months)
    
    ymd_loaded = get_last_features_date(cursor)
    print('{}. Getting visits later than {}.\n'.format(datetime.datetime.now(),ymd_loaded))
    sys.stdout.flush()
    
    cnt = 0
    while (ymd_to_calc.strftime('%Y-%m-%d') > ymd_loaded):
        cnt += 1
        if cnt > 70:
            print('{}. Failed to wait for features to be loaded later than {}. Terminating.\n'.format(datetime.datetime.now(),ymd_loaded) + '*'*60)
            sys.stdout.flush()
            exit(1)
        print('Sleeping for 1000 sec. ymd_to_calc: {}. ymd_loaded: {}.'.format(ymd_to_calc.strftime('%Y-%m-%d'),ymd_loaded))
        sys.stdout.flush()
        time.sleep(1000)
        ymd_loaded = get_last_visits_date(cursor)
    
    print('{} Got visits for {}. Calculating signals for {}'.format(datetime.datetime.now(),ymd_loaded,ymd_to_calc.strftime('%Y-%m-%d')))
    sys.stdout.flush()

2017-06-07 12:44:11.606571: cred scoring started.


In [13]:
def add_months(ymd,months):
    '''ymd is datetime.datetime.'''
    year = ymd.year + (ymd.month + months - 1) / 12
    month = (ymd.month + months - 1) % 12 + 1
    day = ymd.day
    return datetime.datetime(year,month,day)


def check_features_dates(hc, first_calc_ymd, last_calc_ymd):
    return ( len(hc.sql('''select 1 from prod_features_liveinternet.phone_x_feature 
                           where first_calc_ymd = "#first_calc_ymd" and last_calc_ymd = "#last_calc_ymd"
                           limit 1'''
                               .replace( "#first_calc_ymd",first_calc_ymd.strftime('%Y-%m-%d'))
                               .replace( "#last_calc_ymd",last_calc_ymd.strftime('%Y-%m-%d'))
                        ).collect()
                 ) > 0
            )
def main():
    
    acc_months = 2
    # yesterday with 0 time
    #ymd_to_calc   = datetime.datetime.combine(datetime.date.today(),datetime.time(0,0,0)) - datetime.timedelta(days = 1)  
    ymd_to_calc   = datetime.datetime(2017, 6, 5)
    first_calc_ymd = add_months(datetime.datetime(ymd_to_calc.year,ymd_to_calc.month,1), - acc_months)
    
    print('{}. Getting features for ({},{}).'
                    .format(datetime.datetime.now(),first_calc_ymd.strftime('%Y-%m-%d'), ymd_to_calc.strftime('%Y-%m-%d')))
    sys.stdout.flush()
    
    cnt = 0
    while (not check_features_dates(hc, first_calc_ymd, ymd_to_calc)):
        cnt += 1
        if cnt > 70:
            print('{}. Failed to get features to be loaded for ({},{}). Terminating.\n'
                    .format(datetime.datetime.now(),first_calc_ymd.strftime('%Y-%m-%d'), ymd_to_calc.strftime('%Y-%m-%d')) + '*'*60)
            sys.stdout.flush()
            exit(1)
        print('{}.Sleeping for 1000 sec. (first_calc_ymd,ymd_to_calc): ({},{}).'
              .format(datetime.datetime.now(),first_calc_ymd.strftime('%Y-%m-%d'), ymd_to_calc.strftime('%Y-%m-%d')))
        sys.stdout.flush()
        time.sleep(1000)
    
    print('{}. Got features or ({},{}). Calculating cred scores.'
                    .format(datetime.datetime.now(),first_calc_ymd.strftime('%Y-%m-%d'), ymd_to_calc.strftime('%Y-%m-%d')))
    sys.stdout.flush()
main()

2017-06-07 13:12:07.875281. Getting features for (2017-04-01,2017-06-05).
2017-06-07 13:12:09.806471. Got features or (2017-04-01,2017-06-05). Calculating cred scores.


In [27]:
hand_pipe1 = cPickle.load(open('data/cred_scor_classifier_pipe3.pck','r'))
feats                = hand_pipe1[0][1]
categorical_features = hand_pipe1[1][1]
labelencoder         = hand_pipe1[2][1]
target_encoded       = hand_pipe1[3][1]
clf                  = hand_pipe1[4][1]

In [58]:
if True:
    hc.sql('drop table prod_features_liveinternet.phone_cred_scor_tmp')
    os.popen('hadoop fs -mkdir -p /prod_features/liveinternet/phone_cred_scor_tmp').read()
    os.popen('hadoop fs -rm -r /prod_features/liveinternet/phone_cred_scor_tmp/*').read()

    for ind in list('0123456789abcdef'):
        df_batch = (hc.sql('''select * from prod_features_liveinternet.phone_x_feature 
                        where  first_calc_ymd = "#first_calc_ymd" 
                               and last_calc_ymd = "#last_calc_ymd"
                               and substr(md5(phone_num),1,1) = '#ind' '''
                       .replace( "#first_calc_ymd",'2017-04-01')
                       .replace( "#last_calc_ymd",'2017-06-05')
                       .replace( "#ind",ind)
                    )        
                .toPandas()
                 )
        for v in categorical_features:
            df_batch.loc[:,v]  = df_batch.loc[:,v].map(lambda f: labelencoder[v].get(f,-1))
            df_batch.loc[:,v + '_encoded'] = df_batch[v].map(target_encoded[v])
        df_batch['pred'] = clf.predict_proba(df_batch[feats])[:,1]
        df_batch[['phone_num','pred','last_calc_ymd']].to_csv('data/phone_cred_scor_tmp.txt'.format(ind),index = False,header = False,quote_char = '')
        os.popen('hadoop fs -put data/phone_cred_scor_tmp.txt /prod_features/liveinternet/phone_cred_scor_tmp/phone_cred_scor_tmp_{}.txt'.format(ind)).read()
        print('{}. {} handled.'.format(datetime.datetime.now(),ind))

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 500 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction
2017-06-07 17:45:57.831054. 0 handled.
[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 500 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction
2017-06-07 18:01:43.492195. 1 handled.
[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 500 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction
2017-06-07 18:17:27.656690. 2 handled.
[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 500 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction
2017-06-07 18:33:46.585368. 3 handled.
[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 500 models
[LightGBM] [Info] Finished i

In [59]:
if True:
    end_query = '''
    create external table prod_features_liveinternet.phone_cred_scor_tmp
    (
      `phone_num` string,
      `score` float,  
      `ymd` string
      )
    ROW FORMAT DELIMITED
       FIELDS TERMINATED BY ','
        LOCATION
          '/prod_features/liveinternet/phone_cred_scor_tmp'
    ;

    insert overwrite table prod_lookalike.phone_x_segment partition (ymd)
    select 
      phone_num,
      score,
      current_timestamp() as load_dttm,
      'cred_score_1' as segment_nm,
      ymd
    from prod_features_liveinternet.phone_cred_scor_tmp;
    '''

    for q in end_query.split(';'):
        hc.sql(q)

AnalysisException: u'Table not found: prod_features_liveinternet.phone_cred_score; line 1 pos 78'

In [28]:
for v in categorical_features:
    df_batch.loc[:,v]  = df_batch.loc[:,v].map(lambda f: labelencoder[v].get(f,-1))
    df_batch.loc[:,v + '_encoded'] = df_batch[v].map(target_encoded[v])
    
df_batch['pred'] = clf.predict_proba(df_batch[feats])[:,1]

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 500 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


In [43]:
#df_batch[['phone_num','pred','last_calc_ymd']].to_csv('data/phone_cred_scor_tmp.txt'.format(ind),index = False,header = False,quote_char = '')

''

In [49]:
ind = 1
os.popen('hadoop fs -put data/phone_cred_scor_tmp.txt /prod_features/liveinternet/phone_cred_scor_tmp/phone_cred_scor_tmp_{}.txt'.format(ind)).read()



''

In [57]:
os.popen('hadoop fs -rm -r /prod_features/liveinternet/phone_cred_scor_tmp/*').read()
! hadoop fs -du -h /prod_features/liveinternet/phone_cred_scor_tmp

In [55]:
end_query = '''
create external table prod_features_liveinternet.phone_cred_scor_tmp
(
  `phone_num` string,
  `score` float,  
  `ymd` string
  )
ROW FORMAT DELIMITED
   FIELDS TERMINATED BY ','
    LOCATION
      '/prod_features/liveinternet/phone_cred_scor_tmp'
;

insert overwrite table prod_features_liveinternet.phone_cred_score partition (ymd)
select * from prod_features_liveinternet.phone_cred_scor_tmp;
'''

for q in end_query.split(';'):
    hc.sql(q)

DataFrame[result: string]

In [ ]:
create table prod_features_liveinternet.phone_cred_scor_tmp
(
  `phone_num` string,
  `score` float,
  `first_calc_ymd` string, 
  `last_calc_ymd` string
  );